# S-parameter extraction

One thing you might be interested in using MEEP for is to obtain the S-parameters (transmission, reflection, etc.) of your device. This involves manipulating the results of the time-domain simulation.

Back to last Notebook's directional coupler :

In [1]:
# Import meep and mpb (from meep)
import meep as mp
from meep import mpb

# arrays
import numpy as np

# plotting
import matplotlib.pyplot as plt

In [2]:
res = 50        # pixels/μm
three_d = False # 3d calculation?
d = 0.12        # branch separation

gdsII_file = 'coupler.gds'
CELL_LAYER = 0
SOURCE_LAYER = 5
UPPER_BRANCH_LAYER = 31
LOWER_BRANCH_LAYER = 32

t_oxide = 1.0
t_Si = 0.22
t_air = 0.78

dpml = 1
cell_thickness = dpml+t_oxide+t_Si+t_air+dpml
si_zmin = 0

oxide = mp.Medium(epsilon=2.25)
silicon=mp.Medium(epsilon=12)

lcen = 1.55
fcen = 1/lcen
df = 0.2*fcen

cell_zmax =  0
cell_zmin =  0
si_zmax = 0

# read cell size, volumes for source region and flux monitors,
# and coupler geometry from GDSII file
upper_branch = mp.get_GDSII_prisms(silicon, gdsII_file, UPPER_BRANCH_LAYER, si_zmin, si_zmax)
lower_branch = mp.get_GDSII_prisms(silicon, gdsII_file, LOWER_BRANCH_LAYER, si_zmin, si_zmax)

cell = mp.GDSII_vol(gdsII_file, CELL_LAYER, cell_zmin, cell_zmax)
src_vol = mp.GDSII_vol(gdsII_file, SOURCE_LAYER, si_zmin, si_zmax)

In [3]:
PORT1_LAYER = 1
PORT2_LAYER = 2
PORT3_LAYER = 3
PORT4_LAYER = 4
p1 = mp.GDSII_vol(gdsII_file, PORT1_LAYER, zmin=0, zmax=0)
p2 = mp.GDSII_vol(gdsII_file, PORT2_LAYER, zmin=0, zmax=0)
p3 = mp.GDSII_vol(gdsII_file, PORT3_LAYER, zmin=0, zmax=0)
p4 = mp.GDSII_vol(gdsII_file, PORT4_LAYER, zmin=0, zmax=0)

We can include more dummy layers in the GDS where we draw every port of the device :

In [4]:
PORT1_LAYER = 1
PORT2_LAYER = 2
PORT3_LAYER = 3
PORT4_LAYER = 4
p1 = mp.GDSII_vol(gdsII_file, PORT1_LAYER, zmin=0, zmax=0)
p2 = mp.GDSII_vol(gdsII_file, PORT2_LAYER, zmin=0, zmax=0)
p3 = mp.GDSII_vol(gdsII_file, PORT3_LAYER, zmin=0, zmax=0)
p4 = mp.GDSII_vol(gdsII_file, PORT4_LAYER, zmin=0, zmax=0)

Now declare simulation :

In [5]:
geometry = upper_branch+lower_branch

sources = [mp.EigenModeSource(src=mp.GaussianSource(fcen,fwidth=df),
                              size=src_vol.size,
                              center=src_vol.center,
                              eig_band=1,
                              eig_parity=mp.EVEN_Y+mp.ODD_Z, # Save time on simulation by declaring symmetries
                              eig_match_freq=True)]

# Display simulation object
sim = mp.Simulation(resolution=res,
                    eps_averaging=False,
                    subpixel_maxeval=1,
                    subpixel_tol=1,
                    cell_size=cell.size,
                    boundary_layers=[mp.PML(dpml)],
                    sources=sources,
                    geometry=geometry)

We also add monitors at each port; these will record the field at a specific location using Mode Decomposition (https://meep.readthedocs.io/en/latest/Python_User_Interface/#mode-decomposition), allowing us to compute ratios and hence S-parameters :

In [6]:
mode1 = sim.add_mode_monitor(fcen, 0, 1, mp.ModeRegion(volume=p1))
mode2 = sim.add_mode_monitor(fcen, 0, 1, mp.ModeRegion(volume=p2))
mode3 = sim.add_mode_monitor(fcen, 0, 1, mp.ModeRegion(volume=p3))
mode4 = sim.add_mode_monitor(fcen, 0, 1, mp.ModeRegion(volume=p4))

Run simulation (long enough for time not to affect results) :

In [7]:
sim.run(until_after_sources=100)  # We run with a different option, only for some delay after sources are done oscillating

-----------
Initializing structure...
time for choose_chunkdivision = 0.000326157 s
Working in 2D dimensions.
Computational cell is 34.4 x 8 x 0 with resolution 50
     prism, center = (-9.09425,1.41149,0)
          height 0, axis (0,0,1), 174 vertices:
          (-4,0.15,0)
          (-4.108,0.151,0)
          (-4.215,0.152,0)
          (-4.322,0.155,0)
          (-4.429,0.16,0)
          (-4.535,0.165,0)
          (-4.641,0.171,0)
          (-4.747,0.179,0)
          (-4.852,0.187,0)
          (-5.062,0.207,0)
          (-5.167,0.219,0)
          (-5.271,0.231,0)
          (-5.479,0.259,0)
          (-5.582,0.274,0)
          (-5.685,0.29,0)
          (-5.788,0.307,0)
          (-5.891,0.325,0)
          (-5.993,0.343,0)
          (-6.095,0.362,0)
          (-6.197,0.382,0)
          (-6.299,0.403,0)
          (-6.4,0.424,0)
          (-6.501,0.446,0)
          (-6.703,0.492,0)
          (-6.803,0.515,0)
          (-6.904,0.54,0)
          (-7.204,0.615,0)
          (-7.303,0.642,0)


          (10.63,1.623,0)
          (10.533,1.596,0)
          (10.436,1.568,0)
          (10.338,1.541,0)
          (10.144,1.485,0)
          (10.047,1.456,0)
          (9.949,1.427,0)
          (9.658,1.34,0)
          (9.56,1.31,0)
          (9.463,1.281,0)
          (9.366,1.251,0)
          (9.268,1.221,0)
          (9.074,1.161,0)
          (8.878,1.101,0)
          (8.781,1.071,0)
          (8.585,1.011,0)
          (8.487,0.982,0)
          (8.389,0.952,0)
          (8.291,0.923,0)
          (8.193,0.893,0)
          (8.095,0.864,0)
          (7.996,0.835,0)
          (7.898,0.807,0)
          (7.799,0.778,0)
          (7.7,0.75,0)
          (7.601,0.723,0)
          (7.502,0.695,0)
          (7.403,0.668,0)
          (7.303,0.642,0)
          (7.204,0.615,0)
          (6.904,0.54,0)
          (6.803,0.515,0)
          (6.703,0.492,0)
          (6.501,0.446,0)
          (6.4,0.424,0)
          (6.299,0.403,0)
          (6.197,0.382,0)
          (6.095,0.362,0)
          (5.993

          (7.856,-1.315,0)
          (7.662,-1.259,0)
          (7.564,-1.232,0)
          (7.467,-1.204,0)
          (7.37,-1.177,0)
          (7.272,-1.151,0)
          (7.078,-1.099,0)
          (6.98,-1.074,0)
          (6.883,-1.05,0)
          (6.785,-1.025,0)
          (6.687,-1.002,0)
          (6.59,-0.979,0)
          (6.492,-0.956,0)
          (6.394,-0.934,0)
          (6.296,-0.913,0)
          (6.198,-0.893,0)
          (6.099,-0.873,0)
          (6.001,-0.854,0)
          (5.902,-0.835,0)
          (5.804,-0.817,0)
          (5.705,-0.8,0)
          (5.606,-0.784,0)
          (5.408,-0.754,0)
          (5.309,-0.74,0)
          (5.209,-0.728,0)
          (5.11,-0.716,0)
          (5.01,-0.705,0)
          (4.81,-0.685,0)
          (4.709,-0.677,0)
          (4.608,-0.67,0)
          (4.508,-0.664,0)
          (4.407,-0.659,0)
          (4.305,-0.655,0)
          (4.204,-0.652,0)
          (4,-0.65,0)
          (4,-0.15,0)
          (4.108,-0.151,0)
          (4.215,-0.15

Use the monitors to extract S-parameters :

In [8]:
# S parameters
p1_coeff = sim.get_eigenmode_coefficients(mode1, [1], eig_parity=mp.EVEN_Y+mp.ODD_Z).alpha[0,0,0]
p2_coeff = sim.get_eigenmode_coefficients(mode2, [1], eig_parity=mp.EVEN_Y+mp.ODD_Z).alpha[0,0,0]
p3_coeff = sim.get_eigenmode_coefficients(mode3, [1], eig_parity=mp.EVEN_Y+mp.ODD_Z).alpha[0,0,0]
p4_coeff = sim.get_eigenmode_coefficients(mode4, [1], eig_parity=mp.EVEN_Y+mp.ODD_Z).alpha[0,0,0]

# transmittance
p2_trans = abs(p2_coeff)**2/abs(p1_coeff)**2
p3_trans = abs(p3_coeff)**2/abs(p1_coeff)**2
p4_trans = abs(p4_coeff)**2/abs(p1_coeff)**2

print("trans:, {:.2f}, {:.6f}, {:.6f}, {:.6f}".format(d,p2_trans,p3_trans,p4_trans))

MPB solved for omega_1(2.2349,0,0) = 0.684048 after 10 iters
MPB solved for omega_1(2.09576,0,0) = 0.645197 after 7 iters
MPB solved for omega_1(2.09563,0,0) = 0.645161 after 4 iters
MPB solved for omega_1(2.09563,0,0) = 0.645161 after 1 iters
Dominant planewave for band 1: (2.095628,-0.000000,0.000000)
MPB solved for omega_1(2.2349,0,0) = 0.684048 after 10 iters
MPB solved for omega_1(2.09576,0,0) = 0.645197 after 7 iters
MPB solved for omega_1(2.09563,0,0) = 0.645161 after 4 iters
MPB solved for omega_1(2.09563,0,0) = 0.645161 after 1 iters
Dominant planewave for band 1: (2.095628,-0.000000,0.000000)
MPB solved for omega_1(2.2349,0,0) = 0.684048 after 11 iters
MPB solved for omega_1(2.09576,0,0) = 0.645197 after 7 iters
MPB solved for omega_1(2.09563,0,0) = 0.645161 after 4 iters
MPB solved for omega_1(2.09563,0,0) = 0.645161 after 1 iters
Dominant planewave for band 1: (2.095628,-0.000000,0.000000)
MPB solved for omega_1(2.2349,0,0) = 0.684048 after 12 iters
MPB solved for omega_1(2

This checks out with the graph from Notebook 4 (all power going to port 3).

You could verify this for different spacings between the branches, different wavelengths, different waveguides, etc.